In [ ]:
import asyncio
from order_utils import manage_order
import pandas as pd

In [ ]:
from client_utils import ClientSetup

totp = "929198"
setup = ClientSetup()
client = setup.login(totp)
instrument_tokens = [{"instrument_token": "Nifty 50", "exchange_segment": "nse_cm"},]
print("Nifty Current Price: ",client.quotes(instrument_tokens = instrument_tokens, quote_type = "all")[0]['ltp'])

In [ ]:

csv_data = client.scrip_master(exchange_segment="nse_fo")
df = pd.read_csv(csv_data)
df = df[df["pSymbolName"].str.match(r"(?i)^NIFTY$", na=False)]
df.to_csv("nifty-options.csv", index=False)

In [ ]:
tasks = []

for i in range(10):
    tasks.append(asyncio.create_task(manage_order(client, amo="YES")))

await asyncio.gather(*tasks)

In [ ]:
import time
from datetime import datetime, timezone, timedelta
from IPython.display import clear_output
from client_utils import latest_tick

prev_tick = ""
for i in range(30):
    time.sleep(1)
    # print("Fetching latest tick...")
    if not latest_tick:
        continue
    if prev_tick == latest_tick:
        print("No data change, skipping...")
        clear_output(wait=True)
        continue
    print(latest_tick)      
    prev_tick = latest_tick